# Анализ эксперимента run-pid-v1

Анализ работы ПИД-регулятора (аналогичного регулятору Павла) на реальной установке.

Проведено 4 запуска, из которых два были успешными, а два других закончились ошибкой по разным причинам.

Успешные эксперименты:
* 2026-02-05_17-40-14_run-pid-v1
* 2026-02-05_18-02-05_run-pid-v1

In [ ]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.paths import get_experiment_dir

## Загрузка эксперимента

In [ ]:
EXPERIMENT_NAME = "run-pid-v1"
EXPERIMENT_DATE = "2026-02-05"
EXPERIMENT_TIME = "18-02-05"

EXPERIMENT_DIR = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

print(f"Директория эксперимента: {EXPERIMENT_DIR}")
print(f"Существует: {EXPERIMENT_DIR.exists()}")


In [ ]:
config = load_config(EXPERIMENT_DIR / "config.yaml")
pid_config = config.pid

print(f"Эксперимент: {config.experiment_name}")
print(f"\nПараметры PID:")
print(f"  kp = {pid_config.kp}")
print(f"  ki = {pid_config.ki}")
print(f"  kd = {pid_config.kd}")
print(f"  dt = {pid_config.dt}")
print(f"\nУставка: {config.setpoint}")
print(f"Warmup: {config.warmup_steps} шагов, output={config.warmup_output}")

In [ ]:
from nn_laser_stabilizer.pid import PID

pid = PID(
    kp=pid_config.kp,
    ki=pid_config.ki,
    kd=pid_config.kd,
    dt=pid_config.dt,
    min_output=pid_config.min_output,
    max_output=pid_config.max_output,
)

## Парсинг логов соединения

In [ ]:
def parse_connection_log(log_path: Path) -> pd.DataFrame:
    send_pattern = re.compile(r"\[PHASE_SHIFTER\]\s+send:\s+control_output=(\d+)")
    read_pattern = re.compile(r"\[PHASE_SHIFTER\]\s+read:\s+process_variable=(\d+)")
    
    control_outputs = []
    process_variables = []
    
    with open(log_path, "r") as f:
        for line in f:
            send_match = send_pattern.search(line)
            if send_match:
                control_outputs.append(int(send_match.group(1)))
                continue
            
            read_match = read_pattern.search(line)
            if read_match:
                process_variables.append(int(read_match.group(1)))
    
    min_len = min(len(control_outputs), len(process_variables))
    
    return pd.DataFrame({
        "step": range(min_len),
        "control_output": control_outputs[:min_len],
        "process_variable": process_variables[:min_len],
    })

In [ ]:
log_path = EXPERIMENT_DIR / "connection.log"
df = parse_connection_log(log_path)

df["setpoint"] = config.setpoint
df["error"] = df["process_variable"] - df["setpoint"]

print(f"Загружено {len(df)} шагов")
df.head(10)

## Обзор данных

In [ ]:
warmup_steps = config.warmup_steps

df_warmup = df[df["step"] < warmup_steps]
df_work = df[df["step"] >= warmup_steps]

print(f"Warmup: {len(df_warmup)} шагов")
print(f"Рабочая фаза: {len(df_work)} шагов")

In [ ]:
print("Статистика рабочей фазы:")
print(f"\nProcess Variable:")
print(f"  Среднее: {df_work['process_variable'].mean():.2f}")
print(f"  Std: {df_work['process_variable'].std():.2f}")
print(f"  Min: {df_work['process_variable'].min()}")
print(f"  Max: {df_work['process_variable'].max()}")

print(f"\nОшибка (error = pv - setpoint):")
print(f"  MAE: {df_work['error'].abs().mean():.2f}")
print(f"  Среднее: {df_work['error'].mean():.2f}")
print(f"  Std: {df_work['error'].std():.2f}")

print(f"\nControl Output:")
print(f"  Среднее: {df_work['control_output'].mean():.2f}")
print(f"  Std: {df_work['control_output'].std():.2f}")
print(f"  Min: {df_work['control_output'].min()}")
print(f"  Max: {df_work['control_output'].max()}")

## Визуализация

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

ax1 = axes[0]
ax1.plot(df["step"], df["process_variable"], label="Process Variable", alpha=0.8, linewidth=0.5)
ax1.axhline(config.setpoint, color="red", linestyle="--", label=f"Setpoint = {config.setpoint}")
ax1.axvline(warmup_steps, color="orange", linestyle=":", alpha=0.7, label="Конец warmup")
ax1.set_ylabel("Process Variable")
ax1.legend(loc="upper right")
ax1.set_title("Process Variable vs Setpoint")
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(df["step"], df["control_output"], label="Control Output", alpha=0.8, linewidth=0.5, color="green")
ax2.axvline(warmup_steps, color="orange", linestyle=":", alpha=0.7)
ax2.set_ylabel("Control Output")
ax2.legend(loc="upper right")
ax2.set_title("Control Output")
ax2.grid(True, alpha=0.3)

ax3 = axes[2]
ax3.plot(df["step"], df["error"], label="Error", alpha=0.8, linewidth=0.5, color="purple")
ax3.axhline(0, color="red", linestyle="--", alpha=0.5)
ax3.axvline(warmup_steps, color="orange", linestyle=":", alpha=0.7)
ax3.set_ylabel("Error (PV - SP)")
ax3.set_xlabel("Step")
ax3.legend(loc="upper right")
ax3.set_title("Error")
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Компонентный анализ

In [ ]:
P, I, D, U = [], [], [], []

integral = 0.0
prev_error = 0.0
pid.min_output = config.warmup_output
pid.max_output = config.warmup_output

for step, pv in enumerate(df["process_variable"].values):
    error = pv / 10 - config.setpoint / 10

    p = pid.kp * error

    integral = np.clip(
        integral + error * pid.dt * pid.ki,
        pid.min_output,
        pid.max_output,
    )

    derivative = (error - prev_error) / pid.dt
    d = pid.kd * derivative
    prev_error = error

    u = np.clip(
        p + integral + d,
        pid.min_output,
        pid.max_output,
    )

    if step + 1 == config.warmup_steps:
        pid.min_output = config.pid.min_output
        pid.max_output = config.pid.max_output

    P.append(p)
    I.append(integral)
    D.append(d)

df["P"] = P
df["I"] = I
df["D"] = D

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

ax.plot(df["step"], df["P"], label="P")
ax.plot(df["step"], df["I"], label="I")
ax.plot(df["step"], df["D"], label="D")

ax.axvline(
    config.warmup_steps,
    linestyle="--",
    color="gray",
    label="warmup end",
)

ax.set_xlabel("Step")
ax.set_ylabel("Value")
ax.set_title("PID components")
ax.legend()
ax.grid(True)


## Анализ переходного процесса (начало рабочей фазы)

In [ ]:
N_STEPS = 2000

df_transient = df[(df["step"] >= warmup_steps) & (df["step"] < warmup_steps + N_STEPS)]

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

ax1 = axes[0]
ax1.plot(df_transient["step"], df_transient["process_variable"], linewidth=0.8)
ax1.axhline(config.setpoint, color="red", linestyle="--", label=f"Setpoint = {config.setpoint}")
ax1.set_ylabel("Process Variable")
ax1.legend()
ax1.set_title(f"Переходный процесс (первые {N_STEPS} шагов после warmup)")
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(df_transient["step"], df_transient["control_output"], linewidth=0.8, color="green")
ax2.set_ylabel("Control Output")
ax2.set_xlabel("Step")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()